In [1]:
import requests
import json
import re

from collections import Counter

from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI
from openai import OpenAI

client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('optimaldb')
task = get_task(token, print_task=True)

database_url = task['database']

----------- Task description -----------
msg: In a moment you will receive from me a database on three people. It is over 30kb in size. You need to prepare me for an exam in which I will be questioned on this database. Unfortunately, the capacity of my memory is just 9kb. Send me the optimised database
database: https://zadania.aidevs.pl/data/3friends.json
hint: I will use GPT-3.5-turbo to answer all test questions
-----------    ----------    -----------


In [2]:
# Download JSON data from the URL
response = requests.get(database_url)
db = response.json()

In [3]:
def kb_for_json(j, prnt=True):
    # Calculate the size of the variable in kilobytes
    size_in_kilobytes = len(json.dumps(j)) / 1024
    
    # Calculate the total number of words
    word_counts = Counter(json.dumps(j).split())    
    total_words = sum(word_counts.values())
    
    if prnt:
        print(f"Size of the variable: {size_in_kilobytes:.2f} KB for total words: {total_words}")
    
    return size_in_kilobytes
    
kb_before = kb_for_json(db)

Size of the variable: 28.67 KB for total words: 3100


#### Idea is to use gpt-4 model to analyze data for each person and write shorter summary of main information :)

In [4]:
compress_prompt_v1 = f"""Twoją rolą jest zmniejszenie objętości poniżej bazy danych o 70%. 
Zrób to według tych wskazówek:
1) Najpierw przeanalizuj linijka po linijce wszystkie informacje.
2) Następnie bardzo podobne informacje połącz razem w jedno zdanie.
3) W kolejnym kroku przepisz zdania swoimi słowami skracając liczbę słów tylko do tych istotnych.
4) Staraj się nic istotnego nie pominąć.
5) Zwróć wynik tylko jako zdania, które zawsze kończą się kropką `.`
6) Nie używaj żadnych znaków specjalnych (np. \n, @) i cydzysłowiów (np. ",',`,)"""

compress_prompt_v2 = f"""Twoją rolą jest skondensowania in 
Przeanalizuj najpierw cały tekst na temat danej osoby. 
Uprość składnie i napisz najważniejsze informacje krótkimi zdaniami. Nie komplikuj, ma być maksymalnie prosto.
Nie pomiń żadnego istotnego faktu, jak nazwy ulubionych dań, gier itp. 
Nie używaj żadnych znaków specjalnych"""

In [5]:
def compress(db, compress_prompt):
    # ask LLM compressing
    response = client.chat.completions.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": compress_prompt},
            {"role": "user", "content": f"baza danych do przeanalizowania: ###{db}###"}
        ],
        )
    
    db_compress = response.choices[0].message.content
    
    return db_compress

In [6]:
# Create a dictionary
db_new = {}

for person in db.keys():
    
    # using GPT to compress data
    db_compress = compress(db[person], compress_prompt_v2)

    # Remove multiple spaces
    db_compress = re.sub(r'\s+', ' ', db_compress.replace('\n','')).strip()
    
    kb_after = kb_for_json(db_compress)

    # Split the text into sentences based on the presence of a period
    db_compress_list = db_compress.split('. ')    

    db_new[person] = db_compress_list

Size of the variable: 2.04 KB for total words: 252
Size of the variable: 1.11 KB for total words: 135
Size of the variable: 1.44 KB for total words: 174


In [7]:
token = get_auth_token('optimaldb') # need to create new token - longer tan 2 sec
send_answer(token, str(db_new)) # API wanted only string not json file

print("Sending answer: done! :)")

Sending answer: done! :)


#### some additional details

In [8]:
kb_before = kb_for_json(db, prnt=False)
kb_after = kb_for_json(db_new, prnt=False)

print(f'Before: {kb_before}; After: {kb_after}; Compress Ratio: {round(100*kb_after/kb_before,2)}%')

Before: 28.6748046875; After: 4.755859375; Compress Ratio: 16.59%


In [10]:
# Database AFTER:
db_new

{'zygfryd': ['Zygfryd jest programistą JavaScript',
  'Jego program zdobył nagrodę na konferencji technologicznej',
  'Gra na ukulele i hoduje rośliny doniczkowe, w tym storczyki',
  "Lubią łamigłówki logiczne i grał w 'Terra Mystica'",
  'Pokonując pieszo góry, robił zdjęcia na Instagram',
  "Był mistrzem ortografii i zagrał na ulubiony film to 'Matrix'",
  'Uczęszczał na sceny stand-upu i lubi kolor niebieski',
  'Promując zdrowe odżywianie, uwielbia pizzę z ananasem',
  'Uczył się gry na skrzypcach, jest fanem piłki nożnej i ma kolekcję markowych zegarków',
  'Codziennie uczy się nowego języka obcego',
  'Jest założycielem lokalnego klubu programistycznego, a jego ulubiony taniec to tango',
  'Wybiera zieloną herbatę zamiast kawy',
  'Grał w zespole rockowym i ma kolekcję powieści science fiction',
  'Jego hobby to modelarstwo, a szczególnie model Enterprise',
  'Ceni sobie klasykę rockową i interesuje się astronomią',
  'Ma zamiłowanie do aikido i jeździ na rower',
  'Jako mentor p

In [11]:
# Database BEFORE:
db

{'zygfryd': ['Podczas ostatniej konferencji technologicznej, program który stworzył Zygfryd wygrał nagrodę za innowacyjność w użyciu JavaScript.',
  'Wielu nie wie, ale ulubionym instrumentem muzycznym Zygfryda jest ukulele, na którym gra po nocach, kiedy kodowanie na dziś się skończy.',
  'Zygfryd, oprócz programowania, interesuje się również hodowlą roślin doniczkowych, a wśród jego zbiorów można znaleźć rzadki gatunek storczyka.',
  'Łamigłówki logiczne toż to pasja Zygfryda, który często bierze udział w konkursach z tej dziedziny i zdobywa wysokie lokaty.',
  "Kiedy zapytasz go o ulubioną grę planszową, Zygfryd bez wahania odpowie, że jest nią 'Terra Mystica'.",
  'W zeszłym roku Zygfryd odbył pieszą wędrówkę po górach a jego Instagram wypełniony jest zdjęciami z tej podróży.',
  'Mało osób wie, że Zygfryd był kiedyś mistrzem ortografii w szkole podstawowej.',
  'W lokalnym maratonie programistycznym to właśnie aplikacja mobilna zaprojektowana przez Zygfryda zgarnęła pierwsze miejs